In [1]:
import numpy as np
import pandas as pd
import geatpy as ea

In [3]:
df1 = pd.read_csv('data\\y1系数.csv')
coef1 = np.array(df1.values[:, 0]).reshape(1,1120)
param1 = df1.values[:, 1:].reshape(1,1120, 8)
df2 = pd.read_csv('data\\y2系数.csv')
coef2 = np.array(df2.values[:, 0]).reshape(1,1120)
param2 = df2.values[:, 1:].reshape(1,1120, 8)
df3 = pd.read_csv('data\\y3系数.csv')
coef3 = np.array(df3.values[:, 0]).reshape(1,1120)
param3 = df3.values[:, 1:].reshape(1,1120, 8)
X0 = pd.read_csv('data\\STNWR运行数据集.csv')[['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']].values
X0 = np.log(X0 + 1e-20)
X0 = (X0 - np.mean(X0, axis=0)) / np.std(X0, axis=0)
y1 = pd.read_csv('data\\STNWR运行数据集.csv')[['Y1']].values
y1 = np.log(y1 + 1e-20)
y2 = pd.read_csv('data\\STNWR运行数据集.csv')[['Y2']].values
y2 = np.log(y2 + 1e-20)
y3 = pd.read_csv('data\\STNWR运行数据集.csv')[['Y3']].values
y3 = np.log(y3 + 1e-20)

class MyProblem(ea.Problem):  # 继承Problem父类
    def __init__(self):
        name = 'MyProblem'  # 初始化name（函数名称，可以随意设置）
        M = 3  # 优化目标个数
        Dim = 112*8*10  # 初始化Dim（决策变量维数）
        maxormins = [-1, 1, -1]  # 初始化maxormins（目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标）
        varTypes = [0] * Dim  # 初始化varTypes（决策变量的类型，0：实数；1：整数）
        lb = np.where(X0 >= 0, X0 * 0.925, X0 * 1.075)  # 下界规则
        ub = np.where(X0 >= 0, X0 * 1.075, X0 * 0.925)  # 上界规则
        lb = lb.flatten()  # 决策变量下界
        ub = ub.flatten()  # 决策变量上界

        lbin = [1] * Dim  # 决策变量下边界（0表示不包含该变量的下边界，1表示包含）
        ubin = [1] * Dim  # 决策变量上边界（0表示不包含该变量的上边界，1表示包含）
        # 调用父类构造方法完成实例化
        ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb, ub, lbin, ubin)

    def evalVars(self, Vars):
        Vars1 = Vars.reshape(-1,1120,8)
        f11 = np.sum(param1*Vars1, axis=2)+coef1
        f111 = np.exp(f11 * np.std(y1, axis=0) +np.mean(y1, axis=0)-1e-20)
        f22 = np.sum(param2*Vars1, axis=2)+coef2
        f222 = np.exp(f22 * np.std(y2, axis=0) +np.mean(y2, axis=0)-1e-20)
        f33 = np.sum(param3*Vars1, axis=2)+coef3
        f333 = np.exp(f33 * np.std(y3, axis=0) +np.mean(y3, axis=0)-1e-20)       
        f1 = np.sum(f111, axis=1).reshape(-1,1)
        f2 = np.sum(f222, axis=1).reshape(-1,1)
        f3 = np.sum(f333, axis=1).reshape(-1,1)
        ObjV = np.hstack([f1, f2, f3])
        return ObjV

# 实例化问题对象
problem = MyProblem()
# 构建算法
algorithm = ea.moea_NSGA2_templet(
    problem,
    ea.Population(Encoding='RI', NIND=10000),
    MAXGEN=500,  # 最大进化代数
    logTras=20)  # 表示每隔多少代记录一次日志信息，0表示不记录。
# 调整交叉和变异参数
algorithm.recOper.XOVR = 0.5    # 降低交叉概率
algorithm.recOper.Beta = 50     # 增大SBX分布指数
algorithm.mutOper.Pm = 0.5      # 提高变异概率
algorithm.mutOper.Beta = 50     # 增大PM步长
# 求解
res = ea.optimize(algorithm, verbose=True, drawing=0, outputMsg=True, drawLog=False, aveFlag=False, dirName='./0.0750res')
print(res.keys())

gen|  eval  |      gd     |     igd     |      hv     |   spacing   
--------------------------------------------------------------------
 0 | 10000  |     None    |     None    | 2.31520E-03 | 5.14176E+04 
 20| 210000 |     None    |     None    | 4.67935E-03 | 3.00378E+04 
 40| 410000 |     None    |     None    | 5.61884E-03 | 9.63929E+03 
 60| 610000 |     None    |     None    | 6.13073E-03 | 7.38058E+03 
 80| 810000 |     None    |     None    | 6.39481E-03 | 1.37333E+04 
100|1010000 |     None    |     None    | 6.60483E-03 | 1.22137E+04 
120|1210000 |     None    |     None    | 7.41346E-03 | 6.36115E+03 
140|1410000 |     None    |     None    | 8.20823E-03 | 4.00943E+03 
160|1610000 |     None    |     None    | 8.04873E-03 | 4.28531E+03 
180|1810000 |     None    |     None    | 8.29677E-03 | 4.20182E+03 
200|2010000 |     None    |     None    | 8.40430E-03 | 2.86992E+03 
220|2210000 |     None    |     None    | 8.75910E-03 | 2.91935E+03 
240|2410000 |     None    |     No